In [21]:
import os
import pandas as pd
import pandas_gbq
import pytz
import io
from dbfread import DBF

In [23]:
#google cloud big query libraries
from google.cloud import bigquery
from google.oauth2 import service_account

key_path = r"C:\Users\cday\tdm-scenarios-c90ba30c3c5d.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
 
client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

In [56]:
sql = """
SELECT
    mode_of_travel,
    origin_zone_name,
    destination_zone_name,
    day_type,
    day_part,
    data_period,
    o_d_traffic_sample_trip_counts,
    o_d_traffic_calibrated_trip_volume
FROM ut-udot-adap-prod.streetlight_data.udot_personal_spring_2019
WHERE
    ((origin_zone_name BETWEEN '350060_1' AND '350060_7') OR 
     (destination_zone_name BETWEEN '350060_1' AND '350060_7'))
"""
df = client.query(sql).to_dataframe()
df.head()

Forbidden: 403 Access Denied: Table ut-udot-adap-prod:streetlight_data.udot_personal_spring_2019: User does not have permission to query table ut-udot-adap-prod:streetlight_data.udot_personal_spring_2019, or perhaps it does not exist in location US.

Location: US
Job ID: 8bb7a093-3c8e-44c1-b690-28c3ce8d99e1


# Read all Scenario Outputs 

In [24]:
tdmVersions = pd.DataFrame([
    ['v832', '\\\modelace\\ModelAce-E\\1 - TDM\\1 - Official Release (full run)\\v8.3.2\\WF TDM v8.3.2 - 2022-09-21'],
    ['v9',   '\\\modelace\\ModelAce-D\\1 - TDM\\3 - Model Dev\\1 - WF\\2 - Sandbox\\v9.0Beta\WF TDM v9.0 - 2022-12-19']
], columns = (['tdmVersion', 'dirTdmVersion']))

segScenarios = pd.DataFrame([
    ['BY_2019'     ,'_SE19_Net19_Summary_SEGID'          , '_SE19_Net19_1_PA_Route'        , '_SE19_Net19'        ],
    ['BY_2023'     ,'_SE23_Net23_Summary_SEGID'          , '_SE23_Net23_1_PA_Route'        , '_SE23_Net23'        ],
    ['EF_RTP_2032' ,'_ExtForces_SE32_Net32_Summary_SEGID', '_SE32_Net32_1_PA_Route'        , '_SE32_Net32'        ],
    ['EF_RTP_2042' ,'_ExtForces_SE42_Net42_Summary_SEGID', '_SE42_Net42_1_PA_Route'        , '_SE42_Net42'        ],
    ['EF_RTP_2050' ,'_ExtForces_SE50_Net50_Summary_SEGID', '_SE50_Net50_1_PA_Route'        , '_SE50_Net50_EF'     ],
    ['Needs_2032'  ,'_Needs_SE32_Net32_Summary_SEGID'    , '_SE32_Net32_Needs_1_PA_Route'  , '_SE32_Net32_Needs'  ],
    ['Needs_2042'  ,'_Needs_SE42_Net42_Summary_SEGID'    , '_SE42_Net42_Needs_1_PA_Route'  , '_SE42_Net42_Needs'  ],
    ['Needs_2050'  ,'_Needs_SE50_Net50_Summary_SEGID'    , '_SE50_Net50_Needs_1_PA_Route'  , '_SE50_Net50_Needs'  ],
    ['NoBuild_2032','_NoBuild_SE32_Net28_Summary_SEGID'  , '_SE32_Net28_NoBuild_1_PA_Route', '_SE32_Net28_NoBuild'],
    ['NoBuild_2042','_NoBuild_SE42_Net28_Summary_SEGID'  , '_SE42_Net28_NoBuild_1_PA_Route', '_SE42_Net28_NoBuild'],
    ['NoBuild_2050','_NoBuild_SE50_Net28_Summary_SEGID'  , '_SE50_Net28_NoBuild_1_PA_Route', '_SE50_Net28_NoBuild'],
    ['RTP_2032'    ,'_RTP_SE32_Net32_Summary_SEGID'      , '_SE32_Net32_1_PA_Route'        , '_SE32_Net32'        ],
    ['RTP_2042'    ,'_RTP_SE42_Net42_Summary_SEGID'      , '_SE42_Net42_1_PA_Route'        , '_SE42_Net42'        ],
    ['RTP_2050'    ,'_RTP_SE50_Net50_Summary_SEGID'      , '_SE50_Net50_1_PA_Route'        , '_SE50_Net50'        ],
    ['TIP_2028'    ,'_TIP_SE28_Net28_Summary_SEGID'      , '_SE28_Net28_1_PA_Route'        , '_SE28_Net28'        ]
], columns = (['ScenarioName','segSummaryCsvName','routeSummaryDbfName','seSummaryDbfName']))

# Merge all Scenario Outputs

In [153]:
dirWork    = os.getcwd()

lstScenarios = list()
lstSegSummary = list()
lstRouteSummary = list()
lstSESummary = list()
scenarioID = 1

for tdmNum in range(len(tdmVersions)):
    for scenarioNum in range(len(segScenarios)):
        try:
            # get and join all segment summaries
            dirSegSummary = os.path.join(dirWork, tdmVersions.iloc[tdmNum,1] + '\\Scenarios\\' + segScenarios.iloc[scenarioNum,0] + '\\5_AssignHwy\\4_Summaries\\' + tdmVersions.iloc[tdmNum,0] +  segScenarios.iloc[scenarioNum,1] + '.csv')
            dfSegSummary = pd.read_csv(dirSegSummary)
            dfSegSummary['scenarioID'] = scenarioID
            lstSegSummary.append(dfSegSummary)

            # get and join all transit share data


            # get and join all transit route tables
            dirRouteSummary = os.path.join(dirWork, tdmVersions.iloc[tdmNum,1] + '\\Scenarios\\' + segScenarios.iloc[scenarioNum,0] + '\\4_ModeChoice\\3_TransitAssign\\_' + tdmVersions.iloc[tdmNum,0] +  segScenarios.iloc[scenarioNum,2] + '.dbf')
            dfRouteSummary = pd.DataFrame(DBF(dirRouteSummary, load=True))
            dfRouteSummary['scenarioID'] = scenarioID
            lstRouteSummary.append(dfRouteSummary)

            # get and join all se input tables
            dirSESummary = os.path.join(dirWork, tdmVersions.iloc[tdmNum,1] + '\\Scenarios\\' + segScenarios.iloc[scenarioNum,0] + '\\0_InputProcessing\\SE_File_' + tdmVersions.iloc[tdmNum,0] +  segScenarios.iloc[scenarioNum,3] + '.dbf')
            dfSESummary = pd.DataFrame(DBF(dirSESummary, load=True))
            dfSESummary['scenarioID'] = scenarioID
            lstSESummary.append(dfSESummary)
        
            dfScenarios = pd.DataFrame([
                [scenarioID, tdmVersions.iloc[tdmNum,0], segScenarios.iloc[scenarioNum,0]]
            ], columns = (['scenarioID','tdmVersion','scenarioName']))
            lstScenarios.append(dfScenarios)

            scenarioID += 1

        except FileNotFoundError:
            continue


dfSegSummaryMaster = pd.concat(lstSegSummary)
dfRouteSummaryMaster = pd.concat(lstRouteSummary)
dfSESummaryMaster = pd.concat(lstSESummary)
dfScenariosMaster = pd.concat(lstScenarios)

# Create New Scenario Output Data as BigQuery Files

In [184]:
# create GBQ tables
#pandas_gbq.to_gbq(dfScenariosMaster, 'tdm_scenarios_output.scenarios', project_id="tdm-scenarios")
#pandas_gbq.to_gbq(dfSegSummaryMaster, 'tdm_scenarios_output.segment_summary', project_id="tdm-scenarios")
#pandas_gbq.to_gbq(dfRouteSummaryMaster, 'tdm_scenarios_output.route_summary', project_id="tdm-scenarios")
#pandas_gbq.to_gbq(dfSESummaryMastser, 'tdm_scenarios_output.se_table', project_id="tdm-scenarios")


# Append New Scenario Output Data to BigQuery Files

In [181]:
# functions
def update_gbq_table(tableID, df):
    #client.query("DELETE * FROM " + tableID + "WHERE scenarioID > 0")
    job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE",)
     # Make an API request.
    job = client.load_table_from_dataframe(
        df, 
        tableID, 
        job_config
    ) 
    # Wait for the job to complete.
    job.result()

def append_new_rows(tableID, df):
    dfGBQ = client.query("SELECT * FROM " + tableID).to_dataframe()
    dfGBQList = list(set((dfGBQ.scenarioID.tolist())))
    df2 = df[~(df.scenarioID.isin(dfGBQList))]
    return(df2)

In [182]:
# determine new rows to be added to GBQ tables
dfScenariosMaster2 = append_new_rows('tdm-scenarios.tdm_scenarios_output.scenarios', dfScenariosMaster)
dfSegSummaryMaster2 = append_new_rows('tdm-scenarios.tdm_scenarios_output.segment_summary', dfSegSummaryMaster)
dfRouteSummaryMaster2 = append_new_rows('tdm-scenarios.tdm_scenarios_output.route_summary', dfRouteSummaryMaster)
dfSESummaryMaster2 = append_new_rows('tdm-scenarios.tdm_scenarios_output.se_table', dfSESummaryMaster)

In [183]:
# add new scenario rows to GBQ tables
update_gbq_table('tdm-scenarios.tdm_scenarios_output.scenarios', dfScenariosMaster2)
update_gbq_table('tdm-scenarios.tdm_scenarios_output.segment_summary', dfSegSummaryMaster2)
update_gbq_table('tdm-scenarios.tdm_scenarios_output.route_summary', dfRouteSummaryMaster2)
update_gbq_table('tdm-scenarios.tdm_scenarios_output.se_table', dfSESummaryMaster2)

# Notes

In [168]:
dfGBQ = client.query("SELECT * FROM " + 'tdm-scenarios.tdm_scenarios_output.scenarios').to_dataframe()
dfGBQList = list(set((dfGBQ.scenarioID.tolist())))[0:20]
df3 = dfScenariosMaster[~(dfScenariosMaster.scenarioID.isin(dfGBQList))]



In [67]:
job = client.query("DELETE FROM " + 'tdm-scenarios.tdm_scenarios_output.scenarios' + " WHERE scenarioID > 0")
job.result()

Forbidden: 403 Billing has not been enabled for this project. Enable billing at https://console.cloud.google.com/billing. DML queries are not allowed in the free tier. Set up a billing account to remove this restriction.

Location: us-west3
Job ID: e1725aee-d386-469b-b1b6-47656ed242e2


In [ ]:
# update scenarios
table_id_1 = 'tdm-scenarios.tdm_scenarios_output.scenarios'
job = client.load_table_from_dataframe(
    dfScenariosMaster, 
    table_id_1, 
    job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.
table = client.get_table(table_id_1)  # Make an API request.

In [27]:
dfTemp = client.query("SELECT * FROM `tdm-scenarios.tdm_scenarios_output.scenarios` LIMIT 1000").to_dataframe()
dfTemp

,scenarioID,tdmVersion,scenarioName
0,1,v832,BY_2019
1,2,v832,BY_2023
2,3,v832,Needs_2032
3,4,v832,Needs_2042
4,5,v832,Needs_2050
5,6,v832,NoBuild_2032
6,7,v832,NoBuild_2042
7,8,v832,NoBuild_2050
8,9,v832,RTP_2032
9,10,v832,RTP_2042


In [20]:
client.get_table("tdm-scenarios.tdm_scenarios_output.scenarios")  # Make an API request.

Forbidden: 403 GET https://bigquery.googleapis.com/bigquery/v2/projects/tdm-scenarios/datasets/tdm_scenarios_output/tables/scenarios?prettyPrint=false: Access Denied: Table tdm-scenarios:tdm_scenarios_output.scenarios: Permission bigquery.tables.get denied on table tdm-scenarios:tdm_scenarios_output.scenarios (or it may not exist).

In [26]:
table_id_s = 'tdm-scenarios.tdm_scenarios_output.segment_summary'
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
job = client.load_table_from_dataframe(dfSegSummaryMaster, table_id_s, job_config)  # Make an API request.
job.result()  # Wait for the job to complete.
table = client.get_table(table_id_s)  # Make an API request.



In [28]:
display(table)

Table(TableReference(DatasetReference('tdm-scenarios', 'tdm_scenarios_output'), 'segment_summary'))

In [6]:
# Still To Do:
#   - Start seaborn plot comparison code (in new folder?)
#   - Figure out how to "apped" the tables I created to BigQuery!
#       - append both id table and other tables -- bill is showing example